In [ ]:
# install package modisco-lite
# !pip install modisco-lite==2.2.1

In [ ]:
# install package of meme for TOMTOM comparison
# !conda install --quiet --yes -c bioconda meme

In [ ]:
# install package of requests
# !conda install -y conda-forge::requests

In [ ]:
# install package of beautifulsoup4
# !conda install -y anaconda::beautifulsoup4

In [ ]:
# install openpyxl
# !pip install openpyxl

In [1]:
import numpy as np
import h5py
import joblib
import glob
import re
import pandas as pd
import os
import subprocess
import argparse
import requests
import re
from bs4 import BeautifulSoup
from IPython.display import display, HTML



In [2]:
# file path of all the contribution scores, hypothetical contribution scores and onehot encoding of the sequences
file_paths = ['Zhihao_Code/contribution_scores/mesoderm/diff_1.3/BestModel_contr_scores_mesoderm.h5',
              'Zhihao_Code/contribution_scores/cardiac/diff_1.3/BestModel_contr_scores_cardiac.h5',
              'Zhihao_Code/contribution_scores/visceral/diff_1.3/BestModel_contr_scores_visceral.h5',
              'Zhihao_Code/contribution_scores/somatic/diff_1.3/BestModel_contr_scores_somatic.h5',
              'Zhihao_Code/contribution_scores/mesoderm/diff/BestModel_contr_scores_mesoderm.h5',
              'Zhihao_Code/contribution_scores/cardiac/diff/BestModel_contr_scores_cardiac.h5',
              'Zhihao_Code/contribution_scores/visceral/diff/BestModel_contr_scores_visceral.h5',
              'Zhihao_Code/contribution_scores/somatic/diff/BestModel_contr_scores_somatic.h5',
              'Zhihao_Code/contribution_scores/mesoderm/global/BestModel_contr_scores_mesoderm.h5',
              'Zhihao_Code/contribution_scores/cardiac/global/BestModel_contr_scores_cardiac.h5',
              'Zhihao_Code/contribution_scores/visceral/global/BestModel_contr_scores_visceral.h5',
              'Zhihao_Code/contribution_scores/somatic/global/BestModel_contr_scores_somatic.h5']

In [3]:
# read in the data for mesoderm
f = h5py.File(file_paths[0], 'r')
actual_contrib_scores = f['actual_contrib_scores/Contrib_scores'][:]
hyp_contrib_scores = f['hyp_contrib_scores/Contrib_scores'][:]
onehot_samples = f['onehot_samples/cand_x'][:]

In [4]:
actual_contrib_scores[0]

array([[-0.00000000e+00, -0.00000000e+00,  0.00000000e+00,
        -5.11014878e-05],
       [-0.00000000e+00, -6.59241749e-04, -0.00000000e+00,
        -0.00000000e+00],
       [ 0.00000000e+00, -2.70656019e-04, -0.00000000e+00,
        -0.00000000e+00],
       ...,
       [ 0.00000000e+00,  8.04727781e-04,  0.00000000e+00,
        -0.00000000e+00],
       [-0.00000000e+00, -4.86345991e-04,  0.00000000e+00,
         0.00000000e+00],
       [ 7.60875642e-04,  0.00000000e+00,  0.00000000e+00,
        -0.00000000e+00]])

In [5]:
hyp_contrib_scores[0]

array([[-3.23460350e-04, -1.62347438e-04,  1.00675046e-04,
        -5.11014878e-05],
       [-2.94170404e-05, -6.59241749e-04, -5.81847795e-04,
        -7.00870951e-05],
       [ 4.88050311e-04, -2.70656019e-04, -1.01336700e-04,
        -9.37179371e-04],
       ...,
       [ 3.60038335e-04,  8.04727781e-04,  3.29026225e-04,
        -1.11727022e-05],
       [-1.94355671e-04, -4.86345991e-04,  8.26081814e-05,
         1.04566134e-04],
       [ 7.60875642e-04,  3.56152246e-04,  1.96030851e-06,
        -1.88279053e-04]])

In [6]:
# Loop over each file and create the npz files for modisco
for file_path in file_paths:
    with h5py.File(file_path, 'r') as f:
        # Load actual contribution scores
        actual_contrib_scores = f['actual_contrib_scores/Contrib_scores'][:]
        
        # Load hypothetical contribution scores
        hyp_contrib_scores = f['hyp_contrib_scores/Contrib_scores'][:]
        
        # Load one-hot encoded samples
        onehot_samples = f['onehot_samples/cand_x'][:]

        # Check and reshape the data if necessary
        if onehot_samples.shape[1] != 4:
            onehot_samples = np.transpose(onehot_samples, (0, 2, 1))

        if actual_contrib_scores.shape[1] != 4:
            actual_contrib_scores = np.transpose(actual_contrib_scores, (0, 2, 1))

        if hyp_contrib_scores.shape[1] != 4:
            hyp_contrib_scores = np.transpose(hyp_contrib_scores, (0, 2, 1))    

        np.savez(os.path.dirname(file_path) + '/attributions_scores.npz', hyp_contrib_scores)
        
        np.savez(os.path.dirname(file_path) + '/onehot_samples.npz', onehot_samples)
   


In [7]:
%%sh
modisco motifs --help

usage: modisco motifs [-h] [-s SEQUENCES] [-a ATTRIBUTIONS] [-i H5PY] -n
                      MAX_SEQLETS [-l N_LEIDEN] [-w WINDOW] [-o OUTPUT] [-v]

optional arguments:
  -h, --help            show this help message and exit
  -s SEQUENCES, --sequences SEQUENCES
                        A .npy or .npz file containing the one-hot encoded
                        sequences.
  -a ATTRIBUTIONS, --attributions ATTRIBUTIONS
                        A .npy or .npz file containing the hypothetical
                        attributions, i.e., the attributions for all
                        nucleotides at all positions.
  -i H5PY, --h5py H5PY  A legacy h5py file containing the one-hot encoded
                        sequences and shap scores.
  -n MAX_SEQLETS, --max_seqlets MAX_SEQLETS
                        The maximum number of seqlets per metacluster.
  -l N_LEIDEN, --n_leiden N_LEIDEN
                        The number of Leiden clusterings to perform with
                        different r

In [8]:
%%sh
modisco report --help

usage: modisco report [-h] -i H5PY -o OUTPUT [-s SUFFIX] [-m MEME_DB]
                      [-n N_MATCHES]

optional arguments:
  -h, --help            show this help message and exit
  -i H5PY, --h5py H5PY  An HDF5 file containing the output from modiscolite.
  -o OUTPUT, --output OUTPUT
                        A directory to put the output results including the
                        html report.
  -s SUFFIX, --suffix SUFFIX
                        The suffix to add to the beginning of images. Should
                        be equal to the output if using a Jupyter notebook.
  -m MEME_DB, --meme_db MEME_DB
                        A MEME file containing motifs.
  -n N_MATCHES, --n_matches N_MATCHES
                        The number of top TOMTOM matches to include in the
                        report.


In [ ]:
# rund medisco-lite
for file_path in file_paths:

    # Get current path
    dir_main_path = os.getcwd() 
    # Extract the directory from the file path
    dir_path = os.path.dirname(file_path)

    # Change to the directory
    os.chdir(dir_path)

    # Run the modisco motifs command
    subprocess.run([
        'modisco', 'motifs',
        '-s', 'onehot_samples.npz',
        '-a', 'attributions_scores.npz',
        '-n', '50000',
        '-o', 'modisco_test_results.h5'
    ])
    
    # Run the modisco report command
    subprocess.run([
        'modisco', 'report',
        '-i', 'modisco_test_results.h5',
        '-o', 'report/',
        '-s', 'report/'
    ])

    # copy 'html' file one level above
    subprocess.run([
        'cp', 'report/motifs.html', '.'
    ])

    # Run the modisco repor compare command
    subprocess.run([
        'modisco', 'report',
        '-i', 'modisco_test_results.h5',
        '-o', 'report_meme/',
        '-s', 'report_meme/',
        '-m', '../../../../Ying_code/modisco/JASPAR2024_CORE_insects_non-redundant_pfms_meme.txt'
    ])

    # copy 'html' file one level above
    subprocess.run([
        'cp', 'report_meme/motifs.html', 'motifs_meme.html'
    ])

    # come back to the main path
    os.chdir(dir_main_path)


In [8]:
# function to annotate the modisco discovered motifs, adding TFs names and class information, then capture the html report information to a csv file
def add_tf_names_to_html_and_extract_table(html_report, list_output_path, table_output_path):
    def get_transcription_factor_info(jaspar_id):
        url = f"http://jaspar.genereg.net/api/v1/matrix/{jaspar_id}/"
        response = requests.get(url)
        
        if response.status_code == 200:
            data = response.json()
            tf_name = data.get('name')
            tf_class = data.get('class')
            return tf_name, tf_class
        else:
            return None, None

    pattern = r'(?<=<td>)MA\d+\.\d+(?=</td>)'

    in_html = html_report
    out_html = os.path.join(os.path.dirname(in_html), os.path.basename(in_html).replace('.html', '_tfnames.html'))
    tf_name_list = []
    tf_class_list = []

    with open(in_html, 'r') as infile:
        with open(out_html, 'w') as outfile:
            lines = infile.readlines()
            for line in lines:
                matches = re.findall(pattern, line)
                if len(matches) == 1:
                    jaspar_id = matches[0]
                    tf_name, tf_class = get_transcription_factor_info(jaspar_id)
                    tf_name_list.append(f"{jaspar_id} - {tf_name}")
                    tf_class_list.append(tf_class)
                    new_entry = f"{jaspar_id} - {tf_name}"
                    newline = line.replace(jaspar_id, new_entry)
                else:
                    newline = line
                outfile.write(newline)

    print(f"Processed {html_report} and created {out_html}")
    print(tf_name_list, "(", len(tf_name_list), " entries)")

    # Replace None with empty string
    tf_name_list = ["" if i is None else i for i in tf_name_list]
    tf_class_list = ["" if i is None else i for i in tf_class_list]

    # Write the lists to a file
    with open(list_output_path, 'w') as f:
        for name, cls in zip(tf_name_list, tf_class_list):
            f.write(f"{name}\t{cls}\n")

    # Extract table data
    with open(out_html, 'r') as f:
        soup = BeautifulSoup(f, 'html.parser')

    table = soup.find('table', {'class': 'dataframe'})
    rows = table.find_all('tr')

    table_data = []
    headers = [header.text for header in rows[0].find_all('th')]
    
    # Exclude unwanted columns
    columns_to_include = ["pattern", "num_seqlets", "match0", "qval0", "match1", "qval1", "match2", "qval2"]
    columns_index = [headers.index(col) for col in columns_to_include]

    for row in rows[1:]:
        cols = row.find_all('td')
        row_data = [cols[i].text for i in columns_index]
        table_data.append(row_data)

    df = pd.DataFrame(table_data, columns=columns_to_include)
    df.to_csv(table_output_path, index=False)

    # Display the modified HTML content
    # display(HTML(str(soup)))

In [9]:
# loop the dataset and run the function above
all_data = {}

for file_path in file_paths:
    # Extract the directory from the file path
    dir_path = os.path.dirname(file_path)
    folder_name = os.path.basename(dir_path)
    
    # Define the path to the HTML report
    html_report_path = os.path.join(dir_path, 'motifs_meme.html')
    list_output_path = os.path.join(dir_path, 'motifs_meme_tfnames.txt')
    table_output_path = os.path.join(dir_path, 'modisco_report_table.csv')
    
    # Check if the HTML report exists
    if os.path.exists(html_report_path):
        # Run the add_tf_names_to_html function on the HTML report
        add_tf_names_to_html_and_extract_table(html_report_path, list_output_path, table_output_path)

        # Read the generated CSV file into a DataFrame
        df = pd.read_csv(table_output_path)
        all_data[folder_name] = df
    else:
        print(f"HTML report not found at {html_report_path}")

# Write all data to a single Excel file with each sheet named after the folder
output_excel_path = 'merged_modisco_report_tables.xlsx'
with pd.ExcelWriter(output_excel_path) as writer:
    for sheet_name, df in all_data.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)    

Processed Zhihao_Code/contribution_scores/mesoderm/diff_1.3/motifs_meme.html and created Zhihao_Code/contribution_scores/mesoderm/diff_1.3/motifs_meme_tfnames.html
['MA0249.3 - twi', 'MA2227.1 - esg', 'MA0086.3 - sna', 'MA1459.2 - M1BP', 'MA2322.1 - ZIPIC', 'MA2205.1 - CG7928', 'MA2191.1 - amos', 'MA0249.3 - twi', 'MA2268.1 - sage', 'MA2282.1 - SREBP', 'MA2244.1 - Hr39', 'MA1461.2 - sv', 'MA1700.1 - Clamp', 'MA0205.3 - Trl', 'MA2198.1 - Blimp-1', 'MA1700.1 - Clamp', 'MA0205.3 - Trl', 'MA2107.1 - cg'] ( 18  entries)
Processed Zhihao_Code/contribution_scores/cardiac/diff_1.3/motifs_meme.html and created Zhihao_Code/contribution_scores/cardiac/diff_1.3/motifs_meme_tfnames.html
['MA0536.2 - pnr', 'MA1456.2 - Dref', 'MA0529.3 - BEAF-32', 'MA0247.3 - tin', 'MA0253.1 - vnd', 'MA0211.1 - bap', 'MA2299.1 - CG4854', 'MA2322.1 - ZIPIC', 'MA1459.2 - M1BP', 'MA2191.1 - amos', 'MA2303.1 - crp', 'MA2268.1 - sage', 'MA2322.1 - ZIPIC', 'MA2271.1 - sens', 'MA1459.2 - M1BP', 'MA2314.1 - hr3', 'MA1700.1 -

In [11]:
# Extract the directory names for diff and global datasets
diff_1_3_datasets = ['mesoderm', 'cardiac', 'visceral', 'somatic']
diff_datasets = ['mesoderm', 'cardiac', 'visceral', 'somatic']
global_datasets = ['mesoderm', 'cardiac', 'visceral', 'somatic']

# Function to read CSV files and convert data into a structured format
def read_csv_and_extract_data(file_paths, datasets):
    data = {}
    class_info = {}
    for path in file_paths:
        dir_path = os.path.dirname(path)
        dataset = os.path.basename(os.path.dirname(os.path.dirname(path)))
        csv_path = os.path.join(dir_path, 'modisco_report_table.csv')
        class_path = os.path.join(dir_path, 'motifs_meme_tfnames.txt')
        
        # Read the class information from motifs_meme_tfnames.txt
        if os.path.exists(class_path):
            with open(class_path, 'r') as f:
                for line in f:
                    name, cls = line.strip().split('\t')
                    class_info[name] = cls
        
        if os.path.exists(csv_path):
            df = pd.read_csv(csv_path)
            for idx, row in df.iterrows():
                for match_col in ['match0', 'match1', 'match2']:
                    motif_name = row[match_col]
                    qval_col = f'qval{match_col[-1]}'
                    if pd.notna(motif_name):
                        if motif_name not in data:
                            data[motif_name] = {dataset: {qval_col: row[qval_col]}}
                        else:
                            if dataset not in data[motif_name]:
                                data[motif_name][dataset] = {qval_col: row[qval_col]}
                            else:
                                data[motif_name][dataset][qval_col] = row[qval_col]
    return data, class_info

# Extract data from the file paths

diff_1_3_data, diff_1_3_class_info = read_csv_and_extract_data(file_paths[:4], diff_1_3_datasets)
diff_data, diff_class_info = read_csv_and_extract_data(file_paths[4:8], diff_datasets)
global_data, global_class_info = read_csv_and_extract_data(file_paths[8:], global_datasets)

# Merge class info dictionaries
all_class_info = {**diff_1_3_class_info, **diff_class_info, **global_class_info}

# Function to convert the extracted data into a DataFrame
def convert_to_dataframe(data, datasets, class_info):
    all_motifs = sorted([k for k in data.keys() if isinstance(k, str)])
    result_df = pd.DataFrame(index=all_motifs, columns=pd.MultiIndex.from_product([datasets, ['qval0', 'qval1', 'qval2']]))
    
    classes = []
    for motif in all_motifs:
        for dataset in datasets:
            for qval in ['qval0', 'qval1', 'qval2']:
                if dataset in data[motif] and qval in data[motif][dataset]:
                    result_df.loc[motif, (dataset, qval)] = data[motif][dataset][qval]
                else:
                    result_df.loc[motif, (dataset, qval)] = np.nan
        # Add class information
        classes.append(class_info.get(motif, ""))
    
    result_df['class'] = classes
    return result_df

# Convert the extracted data to DataFrames with class information
diff_1_3_df = convert_to_dataframe(diff_1_3_data, diff_1_3_datasets, all_class_info)
diff_df = convert_to_dataframe(diff_data, diff_datasets, all_class_info)
global_df = convert_to_dataframe(global_data, global_datasets, all_class_info)

# Write the original DataFrames to a new Excel file
output_excel_path = 'merged_qvalues_report.xlsx'
with pd.ExcelWriter(output_excel_path, engine='openpyxl') as writer:
    diff_1_3_df.to_excel(writer, sheet_name='diff_1_3_qvalues')
    diff_df.to_excel(writer, sheet_name='diff_qvalues')
    global_df.to_excel(writer, sheet_name='global_qvalues')

print(f"Original merged Excel file saved to {output_excel_path}")

# Function to merge qval columns into one by selecting the lowest value
def merge_qval_columns(df):
    merged_df = pd.DataFrame(index=df.index)
    for cell_type in df.columns.levels[0]:
        if cell_type != 'class':
            qvals = df[cell_type].min(axis=1)  # Select the lowest value from qval0, qval1, and qval2
            merged_df[cell_type] = qvals
    merged_df['class'] = df['class']  # Copy the class column
    return merged_df

# Merge qval columns into one for diff and global DataFrames
diff_1_3_df_merged = merge_qval_columns(diff_1_3_df)
diff_df_merged = merge_qval_columns(diff_df)
global_df_merged = merge_qval_columns(global_df)

# Ensure the correct column order
diff_1_3_df_merged = diff_1_3_df_merged[diff_1_3_datasets + ['class']]
diff_df_merged = diff_df_merged[diff_datasets + ['class']]
global_df_merged = global_df_merged[global_datasets + ['class']]

# Write the merged DataFrames to a new Excel file
output_excel_path_merged = 'merged_single_qvalue_report.xlsx'
with pd.ExcelWriter(output_excel_path_merged, engine='openpyxl') as writer:
    diff_1_3_df_merged.to_excel(writer, sheet_name='diff_1_3_qvalues')
    diff_df_merged.to_excel(writer, sheet_name='diff_qvalues')
    global_df_merged.to_excel(writer, sheet_name='global_qvalues')

print(f"Merged Excel file with single qval columns saved to {output_excel_path_merged}")


Original merged Excel file saved to merged_qvalues_report.xlsx
Merged Excel file with single qval columns saved to merged_single_qvalue_report.xlsx


# 